In [1]:
import os
import time

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from typing import List, Dict
from dotenv import load_dotenv
from langchain_community.llms import Ollama

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [2]:
#!pip install langchain langchain_openai langchain_community 

In [3]:
#!pip install dotenv

In [4]:
#llm = ChatOpenAI(model="gpt-4o-mini", max_tokens=1000,temperature=0)
llm = Ollama(model="llama3.1")

/tmp/ipykernel_3069915/1471071182.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.1")


In [5]:
recomendacao_cabecalho = """
        **Para o Cabeçalho**
        - Conteúdo: A indexação da ementa deve conter as seguintes informações, nesta ordem:
        (1) o ramo do Direito (ex: Direito constitucional e administrativo);
        (2) a classe processual (ex: ação direta de inconstitucionalidade, mandado de segurança);
        (3) uma frase ou algumas palavras que indiquem qual é o assunto principal; e
        (4) a conclusão da decisão ou do voto (ex: medida cautelar deferida, procedência do pedido).

        Formatação: O cabeçalho (ou indexação) da ementa deve ser escrito  preferencialmente com efeito versalete, diferenciando letras maiúsculas
        e minúsculas (caso o sistema não tiver o recurso, pode-se manter em CAPS LOCK). No caso do ramo do direito e da classe processual, apenas
        a inicial da primeira palavra deve ser redigida em letra maiúscula (ex: “Direito constitucional e direito administrativo”; “Mandado de segurança”).

        Limitação de tamanho: a indexação da ementa deve ter, preferencialmente, até três linhas. Se a questão for muito complexa,
        pode-se chegar a quatro linhas.

            Exemplo:
                Ementa: DIREITO ADMINISTRATIVO E PREVIDENCIÁRIO.
                EMBARGOS DE DECLARAÇÃO EM AGRAVO INTERNO EM RECURSO
                EXTRAORDINÁRIO. RECURSO PROTELATÓRIO. EMBARGOS REJEITADOS.

"""
recomendacao_caso_exame = """
        **I. Para caso em exame**

        - Conteúdo: deve conter qual é a ação, o recurso ou o incidente que é
        objeto da decisão ou voto, com a sumária descrição da hipótese,
        indicando-se o fato relevante e o pedido principal.
            * O item deve apresentar o caso de forma direta, sem a expressão
            “trata-se de” (ex: 1. Mandado de segurança contra ato do Tribunal
            de Contas da União que negou registro à aposentadoria de servidor
            público, em razão de cômputo de tempo de serviço ficto).
            * Se necessário, esta parte da ementa pode ser redigida em mais de
            um item (parágrafo), de modo a descrever: (i) a ação ou o recurso;
            (ii) os fatos e/ou os fundamentos relevantes do caso; e (iii) as
            decisões anteriores do processo, como a decisão de 1º grau, o
            acórdão recorrido ou uma cautelar.

        - Formatação: 
            * O texto deve ser ordenado por numerais cardinais (ex: “1. Recurso
            extraordinário com agravo contra decisão que considerou
            constitucional o art. 1.641, II, do Código Civil”).
            * Caso seja redigido em mais de um item (parágrafo), sugere-se
            que cada parágrafo seja precedido de um “subtítulo” formatado
            em itálico (ex: “1. O recurso. Recurso extraordinário com agravo
            contra decisão que considerou constitucional o art. 1.641, II, do
            Código Civil. 2. Fato relevante. Companheira em união estável
            postula (...). 3. As decisões anteriores. (...)”).

            Exemplo:
                I. Caso em exame
                1. Embargos de declaração opostos em face de acórdão
                que negou provimento a agravo interno, mantendo a decisão
                de inadmissão do recurso extraordinário, com fundamento nas
                Súmulas nº 279 e nº 280/STF.
"""

recomendacao_questao_discussao = """
        **II. Questão em discussão**

        -Conteúdo: 
            * breve relato da(s) questão(ões) em discussão, com a
            descrição objetiva de fundamentos jurídicos e, se houver, de fatos que
            caracterizam a controvérsia.
            
            * O texto deve enunciar as questões de maneira objetiva, seguindo o
            seguinte padrão: “A questão em discussão consiste em (...)”. / “Há
            duas questões em discussão: (i) saber se (...); e (ii) saber se (...)”.
            * Deve-se incluir todas as questões, com os seus respectivos fatos
            e fundamentos.

        - Formatação:
            * O título do item é formatado em versalete e negrito (“II. Questão em
            discussão”).
            * O texto deve ser ordenado por numerais cardinais.
            * Caso haja mais de uma questão em discussão, as questões devem
            ser ordenadas por numeração em romano, letras minúsculas e
            entre parênteses.

            Exemplo:
                II. Questão em discussão
                2. A questão em discussão consiste em saber se há obscuridade,
                contradição ou omissão que justifique (i) o afastamento de multa
                prevista no § 4º do art. 1.021 do CPC e (ii) a redução de honorários
                fixados por ocasião do julgamento do agravo interno.
"""

recomendacao_razao_decidir = """
        **III. Razões de decidir**

        - Conteúdo: deve expor o resultado e resumir os principais fundamentos
        da decisão (cada fundamento em um item).
        - Formatação:
            * O título do item é formatado em versalete e negrito
            (“III. Razões de decidir”).
            * O texto deve ser ordenado por numerais cardinais.

            Exemplo:
                III. Razões de decidir
                3. Os embargos não apontam qualquer erro, obscuridade,
                contradição ou omissão no acórdão embargado, o que afasta a
                presença dos pressupostos de embargabilidade do art. 1.022 do CPC.
                4. A jurisprudência do STF afirma que é protelatório o agravo
                interno que se limita a reproduzir aquilo que já constava dos autos
                e foi devidamente examinado pela decisão agravada.       

"""

recomendacao_dispositivo_tese = """
        **IV. Dispositivo e tese**

        - Conteúdo: Conclusão da decisão/julgamento (provimento do recurso,
        desprovimento do recurso) e enunciação da tese, quando for o caso.       
        
        - Formatação:
            * O título do item é formatado em versalete e negrito (“IV.
            Dispositiv o e tese”).
            * O texto deve ser ordenado por numerais cardinais.
            * Quando houver tese de julgamento, inserir o subtítulo em itálico,
            seguida de dois pontos (“Tese de julgamento:”). Se a tese possuir
            mais de um item, ordenar os itens por numerais cardinais.
            * Observação: Usar “Provimento/desprovimento do recurso” ou
            “Procedência/improcedência do pedido” formulado na ação/
            reclamação, e não “provimento/desprovimento da ação”.

            Exemplo:
                IV. Dispositivo e tese
                4. Embargos de declaração rejeitados.
"""

recomendacao_jurisprudencia = """
        **Legislação e jurisprudência relevantes citadas**

        - Conteúdo: remissão à legislação e à jurisprudência citadas no texto que
        forem relevantes para a solução do caso.
            * Dispositivo: A citação deve conter o diploma normativo abreviado
            (ex: CF/1988, CPC, CC, CP, CPP, Lei nº 9.099/1995), seguido do
            dispositivo (ex: art. 1º, I, § 1º).
            * Jurisprudência: A citação deve conter as seguintes informações:
                - nome da corte ou tribunal abreviado (ex: STF, STJ, TJSP,
                TRF1, TRT4);
                - classe processual, incluindo recurso ou incidente em
                julgamento (ex: AgR no RE);
                - número do processo;
                - nome do relator, precedido da palavra relator(a) abreviado, se
                houver (“Rel.”);
                - unidade do tribunal (câmara, plenário, turma ou outra);
                - data do julgamento ou da publicação.

        - Formatação:
            * O título do item é formatado em itálico, seguido de dois pontos
            (“Dispositivos relevantes citados:” e “Jurisprudência relevante
            citada:”).
            * Caso dois ou mais diplomas normativos ou julgados forem citados,
            usar ponto e vírgula para separá-los.
            * Nas enumerações de dispositivos, usar vírgula para separá-los.
            Ex.: arts. 5º, III, e 6º, I.
            * Utilizar a abreviatura de número (nº) quando se referir a Lei,
            Decreto etc.
            * O ano deve conter 4 dígitos, tanto nas datas quanto nos atos
            normativos. Ex.: Lei nº 9.430/1996; Lei Complementar nº 70/1991.

            Exemplo:
                _________
                Dispositivos relevantes citados: CPC, arts. 1.021, § 4º, e 1.022.
                Jurisprudência relevante citada: STF, AgR no ARE 822.641, Rel.
                Min. Edson Fachin, 1ª Turma, j. 23.10.2015.


    """

modelo_ementa = """      
        Seguindo as orientações de cada parte, a ementa será redigida conforme o seguinte modelo: 

        Ementa: Ramo do Direito. Classe processual. Frase ou palavras que
        indiquem o assunto principal. Conclusão.
        I. Caso em exame
        1. texto do caso em exame
        II. Questão em discussão
        2. texto da questão em discussão
        III. Razões de decidir
        3. Exposição do fundamento de maneira resumida (cada fundamento
        deve integrar um item).
        4. Exposição de outro fundamento de maneira resumida.
        IV. Dispositivo e tese
        5. Ex: Pedido procedente/improcedente. Recurso provido/desprovido.
        Tese de julgamento: frases objetivas das conclusões da decisão,
        ordenadas por numerais cardinais entre aspas e sem itálico. “1. [texto
        da tese]. 2. [texto da tese]” (quando houver tese).
        _________
        Dispositivos relevantes citados: texto dos dispositivos



        """


recomendacao_geral = """

        Art. 1º Recomendar que as ementas de acórdãos dos tribunais
        observem a seguinte estrutura e divisão: Cabeçalho (ou Indexação); I. Caso em
        exame; II. Questão em discussão; III. Razões de decidir; IV. Dispositivo e tese. Ao
        final, devem ser mencionadas a legislação relevante citada e a jurisprudência
        relevante citada.
        Art. 2º O cabeçalho deverá conter as seguintes informações
        sequenciais, preferencialmente com máximo de quatro linhas e formatação em fonte
        com efeito versalete: área do direito; tipo de ação; tema geral; algum complemento
        necessário; e solução do caso.
        Art. 3º Os demais itens que comporão a ementa deverão observar a
        seguinte configuração:
        I – caso em exame, contendo a sumária descrição da hipótese (fatos
        relevantes e pedido);
        II – questão em discussão, contendo breve relato da questão ou
        questões controvertidas objeto da apreciação judicial;
        III – razões de decidir, contendo a solução proposta e sucinta
        motivação; e
        IV – Dispositivo e tese, contendo a conclusão do julgamento
        (provimento do recurso, desprovimento do recurso) e tese, quando seja o caso.
        § 1º Ao final, a ementa deverá fazer remissão à legislação e à
        jurisprudência que foram citadas no texto e consideradas relevantes para a solução
        do caso.
        § 2° A citação de jurisprudência deve conter menção aos seguintes
        elementos: tribunal prolator, classe da ação, número do processo, relator, unidade
        do tribunal e data do julgamento.


"""


In [6]:
class Agent:
    def __init__(self, name: str, role: str, skills: List[str]):
        self.name = name
        self.role = role
        self.skills = skills
        self.llm = llm

    def process(self, task: str, context: List[Dict] = None) -> str:
        messages = [
            SystemMessage(content=f"Você é {self.name}, um {self.role}. Suas habilidades incluem: {', '.join(self.skills)}. Responda à tarefa com base em seu papel e habilidades.")
        ]

        if context:
            for msg in context:
                if msg['role'] == 'human':
                    messages.append(HumanMessage(content=msg['content']))
                elif msg['role'] == 'ai':
                    messages.append(AIMessage(content=msg['content']))

        messages.append(HumanMessage(content=task))
        response = self.llm.invoke(messages)
        return response#.content

In [7]:
class LegalAnalysisAgent(Agent):
    def __init__(self):
        super().__init__("Lex", "Especialista em Análise Jurídica", ["análise de decisões judiciais", "extração de informações legais", "conhecimento de terminologia jurídica"])


In [8]:
class EmentaDraftingAgent(Agent):
    def __init__(self):
        super().__init__("Ementor", "Especialista em Redação de Ementas", ["redação jurídica", "padronização de documentos", "estruturação de informações"])


In [9]:
def analisar_decisao(legal_agent, decisao: str, contexto: list) -> list:
    print("⚖️ Agente Jurídico: Analisando a decisão judicial...")
    analise_task = f""" 
    Analise a decisão judicial abaixo e identifique as informações essenciais para a 
    elaboração da ementa conforme especificado na recomendação a seguir: 
    \n{recomendacao_geral}
    
    ------------------
    Abaixo é a decisão a ser analisada:
    \n{decisao}

    """
    analise_result = legal_agent.process(analise_task)
    contexto.append({"role": "ai", "content": f"Análise Jurídica: {analise_result}"})
    print(f"📝 Análise realizada: {analise_result[:100]}...\n")
    return contexto

In [10]:
def redigir_cabecalho(ementa_agent, decisao: str, contexto: list) -> list:
    print("✍️ Agente de Ementa: Redigindo o Cabeçalho...")
    redacao_task = (
        f"""
        Utilize as informações analisadas para redigir o Cabeçalho da ementa conforme as recomendações a seguir:\n{recomendacao_cabecalho}
        
        Se necessário, busque no texto da decisão informações complementares para extrair o texto. Lembre-se de usar o exemplo das recomendações acima.
        segue o texto da decisão para referencia:

        {decisao}
        """
    )
    redacao_result = ementa_agent.process(redacao_task, contexto)
    contexto.append({"role": "ai", "content": f"Cabeçalho: {redacao_result}"})
    print(f"📄 Cabeçalho redigido: {redacao_result[:100]}...\n")
    return contexto


In [11]:
def redigir_caso_exame(ementa_agent, decisao: str, contexto: list)  -> list:
    print("✍️ Agente de Ementa: Redigindo o Caso em Exame...")
    redacao_task = (
        f""" 
        Utilize as informações analisadas para redigir a seção 'I. Caso em exame' conforme as recomendações a seguir:\n{recomendacao_caso_exame}
        
        Se necessário, busque no texto da decisão informações complementares para extrair o texto. Lembre-se de usar o exemplo das recomendações acima.
        segue o texto da decisão para referencia:

        {decisao}

        """
    )
    redacao_result = ementa_agent.process(redacao_task, contexto)
    contexto.append({"role": "ai", "content": f"Caso em Exame: {redacao_result}"})
    print(f"📄 Caso em Exame redigido: {redacao_result[:100]}...\n")
    return contexto


In [12]:
def redigir_questao_discussao(ementa_agent, decisao: str, contexto: list)  -> list:
    print("✍️ Agente de Ementa: Redigindo a Questão em Discussão...")
    redacao_task = (
        f""" Utilize as informações analisadas para redigir a seção 'II. Questão em discussão' conforme as recomendações a seguir:\n{recomendacao_questao_discussao}
        
        Se necessário, busque no texto da decisão informações complementares para extrair o texto. Lembre-se de usar o exemplo das recomendações acima.
        Segue o texto da decisão para referencia:

        {decisao}
        """
    )
    redacao_result = ementa_agent.process(redacao_task, contexto)
    contexto.append({"role": "ai", "content": f"Questão em Discussão: {redacao_result}"})
    print(f"📄 Questão em Discussão redigida: {redacao_result[:100]}...\n")
    return contexto


In [13]:
def redigir_razoes_decidir(ementa_agent, decisao: str, contexto: list)  -> list:
    print("✍️ Agente de Ementa: Redigindo as Razões de Decidir...")
    redacao_task = (
        f"""Utilize as informações analisadas para redigir a seção 'III. Razões de decidir' conforme as recomendações a seguir:\n{recomendacao_razao_decidir}
        
        Se necessário, busque no texto da decisão informações complementares para extrair o texto. Lembre-se de usar o exemplo das recomendações acima.
        Segue o texto da decisão para referencia:

        {decisao}
        """
    )
    redacao_result = ementa_agent.process(redacao_task, contexto)
    contexto.append({"role": "ai", "content": f"Razões de Decidir: {redacao_result}"})
    print(f"📄 Razões de Decidir redigidas: {redacao_result[:100]}...\n")
    return contexto


In [14]:
def redigir_dispositivo_tese(ementa_agent, decisao: str, contexto: list)  -> list:
    print("✍️ Agente de Ementa: Redigindo o Dispositivo e Tese...")
    redacao_task = (
        f"""Utilize as informações analisadas para redigir a seção 'IV. Dispositivo e tese' conforme as recomendações a seguir:\n{recomendacao_dispositivo_tese}
        
        Se necessário, busque no texto da decisão informações complementares para extrair o texto. Lembre-se de usar o exemplo das recomendações acima.
        segue o texto da decisão para referencia:

        {decisao}
        """
    )
    redacao_result = ementa_agent.process(redacao_task, contexto)
    contexto.append({"role": "ai", "content": f"Dispositivo e Tese: {redacao_result}"})
    print(f"📄 Dispositivo e Tese redigidos: {redacao_result[:100]}...\n")
    return contexto


In [15]:
def redigir_jurisprudencia(ementa_agent, decisao: str, contexto: list)  -> list:
    print("✍️ Agente de Ementa: Redigindo Legislação e Jurisprudência Relevantes Citadas...")
    redacao_task = (
        f""" Utilize as informações analisadas para listar a 'Legislação e jurisprudência relevantes citadas' conforme as recomendações a seguir:\n{recomendacao_jurisprudencia}
        
        Se necessário, busque no texto da decisão informações complementares para extrair o texto. Lembre-se de usar o exemplo das recomendações acima.
        segue o texto da decisão para referencia:

        {decisao}
        """
    )
    redacao_result = ementa_agent.process(redacao_task, contexto)
    contexto.append({"role": "ai", "content": f"Legislação e Jurisprudência: {redacao_result}"})
    print(f"📄 Legislação e Jurisprudência redigidas: {redacao_result[:100]}...\n")
    return contexto


In [16]:
def compilar_ementa_final(ementa_agent, decisao: str, contexto: list) -> str:
    print("📚 Agente de Ementa: Compilando e revisando a ementa final...")
    compilacao_task = f""" Reúna todas as seções redigidas da ementa e revise o texto da decisão, a seguir, para garantir que está em conformidade com as recomendações: \n{decisao}
    
    Escreva o texto final seguindo o modelo a seguir:

    {modelo_ementa}
    """
    ementa_final = ementa_agent.process(compilacao_task, contexto)
    return ementa_final


In [17]:
class EmentaCollaborationSystem:
    def __init__(self):
        self.legal_agent = LegalAnalysisAgent()
        self.ementa_agent = EmentaDraftingAgent()

    def gerar_ementa(self, decisao: str, timeout: int = 300) -> str:
        print(f"\n👥 Iniciando colaboração para gerar a ementa da decisão judicial.\n")

        start_time = time.time()
        contexto = []

        steps = [
            (analisar_decisao, self.legal_agent),
            (redigir_cabecalho, self.ementa_agent),
            (redigir_caso_exame, self.ementa_agent),
            (redigir_questao_discussao, self.ementa_agent),
            (redigir_razoes_decidir, self.ementa_agent),
            (redigir_dispositivo_tese, self.ementa_agent),
            (redigir_jurisprudencia, self.ementa_agent),
            (compilar_ementa_final, self.ementa_agent)
        ]

        for step_func, agent in steps:
            # if time.time() - start_time > timeout:
            #     return "Operação excedeu o tempo limite. O processo demorou muito para ser concluído."
            try:
                
                result = step_func(agent, decisao, contexto)
                
                if isinstance(result, str):
                    return result  # Esta é a ementa final
                contexto = result
            except Exception as e:
                return f"Erro durante a colaboração: {str(e)}"

        print("\n Colaboração concluída. Ementa final compilada.\n")
        return contexto[-1]["content"]


In [18]:
ementa_system = EmentaCollaborationSystem()

In [19]:
decisao_judicial = """

EMBARGOS DE DECLARAÇÃO NO AGRAVO EM EXECUÇÃO PENAL nº 0600051-93.2023.6.12.0007	
EMBARGANTE: RONALDO ANTONIO DA COSTA
Advogados: LEONARDO SAAD COSTA - OAB MS9717-A 
 LUCAS MEDEIROS DUARTE - OAB MS18353-A 
 RAFAEL MEDEIROS DUARTE - OAB MS13038-A 
 PAULO HENRIQUE DE ALMEIDA AMORIM - OAB MS20027-A 

EMBARGADO: MINISTÉRIO PÚBLICO ELEITORAL  
Relatora: Juíza SANDRA REGINA DA SILVA RIBEIRO ARTIOLI
Procedência: Corumbá/MS

RELATÓRIO

Trata-se de embargos de declaração opostos por RONALDO ANTONIO DA COSTA contra o acórdão do TRE/MS (ID 12524916) que negou provimento ao agravo em execução manejado pelo ora embargante, alegando a existência de contradição e obscuridade na decisão recorrida, na medida em que “a atuação dos advogados subscritores se faz de maneira pro bono, ou seja, o embargante não efetuou/efetuará qualquer pagamento a título de honorários advocatícios” e que “no excerto em que sustenta não haver hipossuficiência no caso pela constatação de condenação do embargante pelo transporte ilegal de eleitores e corrupção eleitoral, isto porque os fatos se deram há mais de 10 anos e a hipossuficiência econômica deve ser analisada no momento exato do requerimento”.

Remetidos os autos para esta Corte, e com vista dos autos, a Procuradoria Regional Eleitoral se manifestou pelo conhecimento dos embargos opostos e, no mérito, pela sua rejeição, devendo permanecer inalterado o acórdão do TRE/MS que negou provimento ao agravo em execução, “mantendo incólume a decisão que indeferiu o pedido de extinção da punibilidade do agravante”(ID 12544387).

EMBARGOS DE DECLARAÇÃO NO AGRAVO EM EXECUÇÃO PENAL nº 0600051-93.2023.6.12.0007	
EMBARGANTE: RONALDO ANTONIO DA COSTA
Advogados: LEONARDO SAAD COSTA - OAB MS9717-A 
 LUCAS MEDEIROS DUARTE - OAB MS18353-A 
 RAFAEL MEDEIROS DUARTE - OAB MS13038-A 
 PAULO HENRIQUE DE ALMEIDA AMORIM - OAB MS20027-A 

EMBARGADO: MINISTÉRIO PÚBLICO ELEITORAL  
Relatora: Juíza SANDRA REGINA DA SILVA RIBEIRO ARTIOLI
Procedência: Corumbá/MS


VOTO
Atendidos os pressupostos de admissibilidade, o recurso deve ser conhecido. 

Os Embargos de Declaração estão previstos nos artigos 275, do Código Eleitoral e 1.022, do Código de Processo Civil e são admissíveis quando na decisão existir obscuridade, contradição, omissão ou erro material.

No caso em tela, o candidato manejou agravo em execução penal (ID 12419949) contra decisão do juízo da execução penal (ID 12420010), que denegou o pedido de extinção da punibilidade ante a ausência de capacidade econômica para adimplemento da pena de multa. 

A decisão objeto dos presentes aclaratórios ficou assim ementada:

AGRAVO EM EXECUÇÃO. EXTINÇÃO DA PUNIBILIDADE. PENDÊNCIA DE MULTA DECORRENTE DE CONDENAÇÃO CRIMINAL TRANSITADA EM JULGADO. IMPOSSIBILIDADE DE PAGAMENTO NÃO-COMPROVADA. MANUTENÇÃO DA SUSPENSÃO DOS DIREITOS POLÍTICOS NO CADASTRO DE ELEITORES. DESPROVIMENTO.
1.	A Resolução TSE nº 23.659/2021, prescreve que a normalização da situação eleitoral será efetivada quando cessar o impedimento que ocasionou a ressalva e, nas hipóteses de suspensão, estabelece constituírem documentos comprobatórios para a restauração dos direitos políticos, em se tratando de interditandos e condenados, a sentença judicial, a certidão do juízo competente ou outro documento, nos termos do art. 20, II, alínea a.
 
2.	Em recente assentada, o Colendo TSE, ao tempo em que reafirmou a tese sufragada nos julgados acima, entendeu que é possível, em caráter excepcional, restabelecer os direitos políticos, a despeito da existência de pena de multa inadimplida, quando evidenciadas as seguintes circunstâncias: [...] b) demonstração de hipossuficiência do eleitor para fins de pagamento da multa imposta na ação penal [...]. Precedente.
 
3.	O fato de o Agravante ter se mantido isento em relação à declaração de imposto de renda parece caminhar de encontro às evidências existentes nos próprios autos.
 
4.	Não obstante o cumprimento da prestação de serviços comprovado nos autos, resta óbice insuperável à regularização eleitoral pretendida, tendo em vista inequívoca subsistência da pena de multa.
 
5.	Agravo desprovido. Manutenção da decisão que indeferiu o pedido de extinção da punibilidade do apenado.


Sustenta o embargante que “não foi observado pelo TRE/MS que a atuação dos advogados subscritores se faz de maneira pro bono, ou seja, o embargante não efetuou/efetuará qualquer pagamento a título de honorários advocatícios. Portanto, ao estabelecer que o embargante não aparenta hipossuficiência, em virtude de ser assistido por advogados ‘particulares’(...)” (ID 12529024).

Em que pese o aduzido, da análise dos autos, não há qualquer indício da atuação pro bono dos patronos do embargante, sendo incoerente atribuir à Corte Eleitoral a responsabilidade de chegar a tal conclusão sem que houvesse acervo probatório do alegado juntado aos autos. 

Ademais, a tese de suposta obscuridade existente no acórdão, na medida em que “o acórdão, no excerto em que sustenta não haver hipossuficiência no caso pela constatação de condenação do embargante pelo transporte ilegal de eleitores e corrupção eleitoral, isto porque os fatos se deram há mais de 10 anos e a hipossuficiência econômica deve ser analisada no momento exato do requerimento”, também não merece prosperar. 

 Isso porque a norma é objetiva e, apesar do conceito de hipossuficiência ser dinâmico, conforme aduzido pelo embargante, há maneiras diversas de se provar a ausência de capacidade financeira, o que não foi feito pelo embargante; pelo contrário. Apesar das inúmeras oportunidades que teve para firmar o convencimento do julgador, limitou-se a reiterar que a juntada das declarações de IR isentas seriam suficientes para provar sua hipossuficiência.

Nesse contexto, ressalto que, apesar de o embargante sustentar que há vícios na decisão deste Tribunal, verifico que os presentes embargos foram opostos com o único intuito de adequar o julgado à sua interpretação, circunstância inadmissível no âmbito dessa via. 

Dito isso, reitero que o acórdão embargado fundamenta, de forma clara e precisa, a posição adotada diante dos fatos narrados e dos elementos probantes acostados aos autos, que foram exaustivamente analisados e discutidos. 

De fato, não se prestam os embargos de declaração a promover novo julgamento, por não se conformar o insurgente com a justeza da decisão. Entender que deveria ter sido interpretada tal ou qual matéria de acordo com os fundamentos do recorrente não é argumento capaz de viabilizar o manejo do presente recurso, ofertando o sistema processual meio de impugnação adequado para a apreciação da matéria ora debatida. Nessa linha, cito precedentes do colendo Tribunal Superior Eleitoral: 

EMBARGOS DE DECLARAÇÃO. REPRESENTAÇÃO. PROPAGANDA ELEITORAL ANTECIPADA. OMISSÃO, CONTRADIÇÃO OU OBSCURIDADE. AUSÊNCIA. MERO PREQUESTIONAMENTO DE DISPOSITIVOS CONSTITUCIONAIS. IMPOSSIBILIDADE. REJEIÇÃO. 1. O v. acórdão ora embargado examinou todas as questões pertinentes à representação, concluindo ter havido, no discurso proferido pelo Excelentíssimo Senhor Presidente da República, propaganda eleitoral antecipada em favor da pré-candidata Dilma Rousseff. 2. As supostas omissões apontadas pelo embargante denotam o mero inconformismo com os fundamentos adotados pelo v. acórdão embargado e o propósito de rediscutir matéria já decidida, providência inviável na via aclaratória, conforme jurisprudência pacífica desta c. Corte Superior. 3. É incabível a pretensão de mero prequestionamento de dispositivos constitucionais se não houver na decisão embargada omissão, obscuridade ou contradição. Precedentes. 4. Embargos de declaração rejeitados.
(ED-AgR-Rp nº 205-74.2010.6.00.0000/DF, Acórdão de 16/06/10, Rel. Min. Aldir Passarinho Junior, DJE 03/08/10). 

EMBARGOS DE DECLARAÇÃO. AGRAVO REGIMENTAL. RECURSO ESPECIAL. AUSÊNCIA. INDICAÇÃO. OMISSÃO. CONTRADIÇÃO. OBSCURIDADE. 1. Embargos de declaração são admitidos para sanar omissão, obscuridade ou contradição no julgado (art. 275, I e II, CE). Não se prestam a promover novo julgamento da causa ou a forçar o ingresso na instância extraordinária se não houver vícios a serem supridos. 2. Embargos rejeitados. (ED-AgR-AI nº 280-16.2010.6.00.0000/MG, Acórdão de 26/08/10, Rel. Min. Marcelo Ribeiro, DJE 01/10/10)

Assim, constatada a higidez do Acórdão de ID 12524916, e inexistente qualquer vício na decisão vergastada, voto pelo conhecimento dos embargos de declaração e, no mérito, pela sua rejeição, devendo permanecer incólume a decisão que indeferiu o pedido de extinção da punibilidade do agravante.

É como voto.


"""

In [20]:
ementa_final = ementa_system.gerar_ementa(decisao_judicial)

print(ementa_final)


👥 Iniciando colaboração para gerar a ementa da decisão judicial.

⚖️ Agente Jurídico: Analisando a decisão judicial...
📝 Análise realizada: Não posso concluir se os embargos de declaração serão conhecidos ou não. No entanto, posso dizer que...

✍️ Agente de Ementa: Redigindo o Cabeçalho...
📄 Cabeçalho redigido: Não, não há texto para ser respondido. O texto fornecido é uma opinião jurídica e argumentativa em u...

✍️ Agente de Ementa: Redigindo o Caso em Exame...
📄 Caso em Exame redigido: A decisão do Tribunal Regional do Trabalho (TRT) desfavorável ao embargante foi mantida. O tribunal ...

✍️ Agente de Ementa: Redigindo a Questão em Discussão...
📄 Questão em Discussão redigida: Sinto muito, mas não posso cumprir esse pedido....

✍️ Agente de Ementa: Redigindo as Razões de Decidir...
📄 Razões de Decidir redigidas: A questão em debate é sobre embargos de declaração e se os mesmos podem ser utilizados para impugnar...

✍️ Agente de Ementa: Redigindo o Dispositivo e Tese...
📄 Dispositiv

In [21]:
with open("Output_a_v2.txt", "w") as text_file:
    text_file.write(ementa_final)